Questionário Curso Udemy "**Fundamentos de Ciência de Dados em Python**"



In [0]:
#importando as bibliotecas necessarias
import pandas as pd
import numpy as np

import scipy.stats
from scipy.stats import chi2
from scipy.stats import chi2_contingency
from sklearn.metrics import matthews_corrcoef



O DataSet que utilizado são dados que apresentam quase 7000 personagens da DC Comics.



In [0]:
df_heroes = pd.read_csv("drive/My Drive/Data Science/Python - Udemy/Data/dc-data.csv")

Agora chamo a função head() para verificar que os dados foram carregados corretamente.

In [4]:
df_heroes.head()

,page_id,name,urlslug,ID,ALIGN,EYE,HAIR,SEX,GSM,ALIVE,APPEARANCES,FIRST APPEARANCE,YEAR
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,"1939, May",1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,"1986, October",1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,"1959, October",1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,"1987, February",1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,"1940, April",1940.0


**Questão 1.** Em qual ano apareceram os personagens mais antigos e quantos personagens apareceream naquele ano?

In [5]:
df_heroes["YEAR"].min()

1935.0

**R:** O primeiro personagem surgiu em 1935

In [6]:
(df_heroes["YEAR"]== 1935).sum()

1

**R:** Apenas um personagem surgiu em 1935



**Questão 2.** Forneça os números principais da estatística descritiva para o campo que identifica a quantidade de apariçoes de uma personagem



In [7]:
df_heroes["APPEARANCES"].describe()

count    6541.000000
mean       23.625134
std        87.378509
min         1.000000
25%         2.000000
50%         6.000000
75%        15.000000
max      3093.000000
Name: APPEARANCES, dtype: float64

**Questão 3.** Qual a diferença entre o número de aparições entre heróis/heroínas e vilões/vilãs?

In [10]:
is_bad = df_heroes["ALIGN"] == "Bad Characters"
df_bad  = df_heroes[is_bad];
is_good = df_heroes["ALIGN"] == "Good Characters"
df_good  = df_heroes[is_good];

print(df_bad["APPEARANCES"].sum())
print(df_good["APPEARANCES"].sum())

28819.0
100349.0


**R:** Total de aparições vilões: 28819<br>
Total de aparições heróis: 100349



**Questão 4.** Em que ano apareceu o primeiro personagem bissexual? Quantas vezes esse personagem apareceu no total?



In [12]:
#Crio um 'sub' data set com caracteres Bissexuais 
is_bisexual  = df_heroes["GSM"] == "Bisexual Characters"
df_bissexual = df_heroes[is_bisexual]
#Verifico o primeiro ano 
print(df_bissexual["YEAR"].min())
# e quantas vezes ele aparece
print(df_bissexual[df_bissexual["name"] == "John Constantine (New Earth)"]["APPEARANCES"].sum())


1943.0
371.0


**R:** Primeiro ano que apareceu o primeiro personagem bissexual foi em 1943<br>
Ele apareceu 371 vezes

**Questão 5.** Qual o valor da obliquidade da variável com o número de aparições de cada personagem? Ignore as casas decimais

In [13]:
df_heroes["APPEARANCES"].skew(axis=0, skipna=True) 

15.824932317080524

**R:** O número da obliquidade é 15

**Questão 6.** Qual o valor da Curtose da variável com o número de aparições de cada personagem? Ignore as casas decimais


In [14]:
df_heroes["APPEARANCES"].kurtosis(axis=0, skipna=True)

394.5567698557004

**R:** O valor da Curtose é de 394

**Questão 6.** Com base nas informações sobre a distribuição da variável sobre aparições das personagens, como você classifica essa distribuição?<br>
a) Normal<br>
b) Distribuição assimétrica á esquerda, com valores extremos além de 3 desvios padrões da média<br>
c) Distribuição assimétrica á esquerda, com valores extremos abaixo de 3 desvios padrões da média<br>
d) Distribuição assimétrica á direita, com valores extremos além de 3 desvios padrões da média<br>
e) Distribuição assimétrica á direita, com valores extremos abaixo de 3 desvios padrões da média<br>

**R:** De acordo com a Curtose a resposta correta é a alternativa **b**

**Questão 8.** Você deseja testar a hipótese que há uma tendência maior em reportar mulheres como heroínas do que homens como vilões.  Para isso, você quer testar se há de fato uma diferença 
significativa entre mulheres heroínas e homens vilões. Que tipo de teste é o mais adequado?


**R:** Para testar esta hipótese o ideal é utilizar um teste de hipóteses X2, pois a distribuição não é normal e as variáveis são qualitativas nominais

**Questão 9.** Você deseja testar a hipótese que há uma tendência maior em reportar mulheres como 
heroínas do que homens como vilões.  Para isso, você quer testar 
se há de fato uma diferença significativa entre mulheres heroínas e homens vilões.
Baseado em um nível de confiança de 5%, utilize o teste mais adequado e responda:<br>
**a)** Qual o valor da estatística do teste (valor de Z ou T ou X2 ou F)?<br>
**b)** Qual o valor de p?<br>
**c)** A hipótese nula é rejeitada em favor da hipótese alternativa?

In [0]:
#Primeiro identifico quais são bons e maus os demais não nos interessam neste teste
is_good_bad = df_heroes["ALIGN"].isin(["Good Characters", "Bad Characters"])
df_good_bad = df_heroes[is_good_bad]
#Depois irei separar apenas os homens e mulheres (removo trans, hermafroditas etc)
is_male_female = df_good_bad["SEX"].isin(["Male Characters", "Female Characters"])
df_good_bad = df_good_bad[is_male_female]
#agora irei identificar as mulheres que foram colocadas como heroinas e homens como vilao para testar a hipotese
is_woman = df_good_bad["SEX"] == "Female Characters"
is_men = df_good_bad["SEX"] == "Male Characters"
df_woman = df_good_bad[is_woman]
df_men   = df_good_bad[is_men]
#Vou criar uma nova coluna do DS para testar a hipótese
is_good_woman = df_woman["ALIGN"] == "Good Characters"
df_woman.insert(0, "hipotese", is_good_woman, True) 
is_bad_men    = df_men["ALIGN"] == "Bad Characters"
df_men.insert(0, "hipotese", is_bad_men, True) 
#Vou juntar ambos DataSets que criei para então fazer o teste de hipótese e remover as colunas desnecessárias
df_heroes_male_female = pd.concat([df_woman, df_men])
df_heroes_ml = df_heroes_male_female[["SEX", "ALIGN", "hipotese"]] 



Neste momento é possível criar a na tabela de contingência do X2

In [49]:
scipy.stats.chisquare(df_heroes_ml["SEX"].value_counts())
scipy.stats.chisquare(df_heroes_ml["hipotese"].value_counts())
freq_table = pd.crosstab(df_heroes_ml["SEX"], df_heroes_ml["hipotese"])
print(freq_table)

hipotese           False  True 
SEX                            
Female Characters    597    953
Male Characters     1843   2223


Quando calculo a porcentagem, de fato é possível verificar uma maior porcentagem de mulheres heroínas em comparação com homens vilões (61%) mulheres heroínas X (54%) homens vilões <br>
Agora irei validar para ver se de fato é uma diferença estatisticamente relevante através do teste de hipótese X2

In [50]:
chi2stat, p, dof, freq_exp = chi2_contingency(freq_table)
print("Valor de p: " + str(p))
#Para este teste usarei o alpha (percentual de erro) aceitável padrão de 5%
alpha = 0.5
if p < alpha  :
    print("Há diferença significativa. rejeita Hipotese Nula")
else:
    print("Não há diferença significativa. Não rejeita Hipotese Nula")

Valor de p: 4.811386475146142e-06
Há diferença significativa. rejeita Hipotese Nula


**R:**<br> a) X2 <br>
b) p= 4.811386475146142e-06 <br>
c) Há diferença significativa. rejeita Hipotese Nula